### torch.nn.Conv2d


    torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)


    padding:`valid`不做padding,`same`pad到最后一个像素点,步长=1时，输出与输入尺寸一致

    dilation:空洞卷积，大于1时增大卷积范围

    groups:把通道分成多组，默认groups=1,深度可分离卷积就是把groups设置成大于1的一个数, 它必须被in_channels和out_channels整除
    groups>1时计算量有下降


同理，和dropout一样，卷积在torch.nn.functional.conv2d中也有一个函数

    torch.nn.functional.conv2d(input, weight, bias=None, stride=1, padding=0, dilation=1, groups=1) → Tensor

In [1]:
import torch
conv_layer = torch.nn.Conv2d(2, 4, 3, padding='same')
for i in conv_layer.named_parameters():
    print(i)

('weight', Parameter containing:
tensor([[[[-0.1002,  0.1696,  0.0249],
          [-0.2027, -0.0728,  0.2135],
          [ 0.0079,  0.0401,  0.0857]],

         [[-0.1457, -0.0404,  0.1309],
          [-0.0901,  0.2190,  0.2311],
          [-0.0051, -0.0506, -0.1456]]],


        [[[ 0.1596,  0.0528,  0.1407],
          [ 0.0526,  0.1839, -0.0605],
          [-0.0147,  0.0170, -0.0121]],

         [[ 0.0620,  0.1423, -0.0495],
          [ 0.1412,  0.1493, -0.2019],
          [ 0.1619, -0.2087, -0.0874]]],


        [[[-0.0257, -0.0613,  0.1271],
          [ 0.1254, -0.2321,  0.0952],
          [ 0.0246, -0.1568,  0.1272]],

         [[-0.2184,  0.0109, -0.1547],
          [ 0.0515,  0.1314, -0.0315],
          [ 0.0297, -0.0016, -0.2111]]],


        [[[-0.0048, -0.0320, -0.0226],
          [ 0.1520,  0.1687, -0.2061],
          [-0.1857, -0.1055,  0.0010]],

         [[ 0.0780, -0.1275, -0.1091],
          [-0.0564,  0.0925, -0.1264],
          [ 0.0093, -0.0966, -0.2343]]]], requires

In [2]:
conv_layer.weight.size()  # 输出通道*输入通道*卷积核大小(3*3)

torch.Size([4, 2, 3, 3])

In [3]:
conv_layer.bias.size()  # 输出通道相关

torch.Size([4])

point-wise convolution:1*1卷积,不考虑局部关联性，只对每个通道上加权求和，相当于一个MLP

depth-wise convolution:groups > 1，并没有把所有通道混合，减少了计算量

In [4]:
# 把卷积分成两部分，每部分输入通道为1，输出通道为2，最后再拼起来
conv_layer = torch.nn.Conv2d(2, 4, 3, padding='same', groups=2)
print(conv_layer.weight.size())
print(conv_layer.bias.size())

torch.Size([4, 1, 3, 3])
torch.Size([4])


In [5]:
# 上述过程等价于
conv_layer1 = torch.nn.Conv2d(1, 2, 3, padding='same')
conv_layer2 = torch.nn.Conv2d(1, 2, 3, padding='same')
print(conv_layer1.weight.size())
print(conv_layer1.bias.size())
print(conv_layer2.weight.size())
print(conv_layer2.bias.size())

torch.Size([2, 1, 3, 3])
torch.Size([2])
torch.Size([2, 1, 3, 3])
torch.Size([2])


### 卷积残差模块算子融合

源自论文RepVGG: Making VGG-style ConvNets Great Again
![structure](img_conv2d/RepVGG.png)

In [6]:
import time
import torch
import torch.nn.functional as F
import torch.nn as nn

In [7]:
in_channels = 2
out_channels = 2
kernel_size = 3
w = 9
h = 9
x = torch.ones(1, in_channels, w, h)  # B, C, W, H

res_block = k=3 conv + k=1 conv + input

In [8]:
# 原生写法
t1 = time.time()
conv_2d = nn.Conv2d(in_channels, out_channels, kernel_size, padding='same')
conv_2d_pointwise = nn.Conv2d(in_channels, out_channels, 1)
result1 = conv_2d(x) + conv_2d_pointwise(x) + x
t2 = time.time()
print("原生写法耗时：",t2-t1)

原生写法耗时： 0.00099945068359375


In [9]:
# 算子融合
# 把point-wise卷积和x本身都写成3*3的卷积
# 最终把三个卷积写成一个卷积
# 1) 改造
pointwise_to_conv_weight = F.pad(conv_2d_pointwise.weight, [1,1,1,1,0,0,0,0])  # 2*2*1*1 -> 2*2*3*3
conv_2d_for_pointwise = nn.Conv2d(in_channels, out_channels, kernel_size, padding='same')
conv_2d_for_pointwise.weight = nn.Parameter(pointwise_to_conv_weight)
conv_2d_for_pointwise.bias = nn.Parameter(conv_2d_pointwise.bias)

'''
把x本身编成3x3卷积
#1.首先要是point-wise卷积（不考虑相邻点关联性）
#2.只要考虑第一个通道，不考虑其他通道（不考虑通道关联性）

对于2x2x3x3的卷积核，可以想象，就是4个3x3的卷积核，每个3x3卷积至少都是中间为1，周围是0
由于只考虑第一个通道，第一个3x3就是如上面所述的point-wise矩阵
第二个3x3就是全0矩阵，第三个也是全0，第四个是point-wise矩阵（输出第二通道只考虑输入第二通道）
意思就是
输出通道1=输入通道1*stars+输入通道2*zeros
输出通道2=输入通道2*stars+输入通道1*zeros
'''

zeros = torch.unsqueeze(torch.zeros(kernel_size, kernel_size), 0)
stars = torch.unsqueeze(F.pad(torch.ones(1,1), [1,1,1,1]), 0)
stars_zeros = torch.unsqueeze(torch.cat((stars, zeros), 0), 0)
zeros_stars = torch.unsqueeze(torch.cat((zeros, stars), 0), 0)
identity_to_conv_weight = torch.cat((stars_zeros, zeros_stars), 0)
identity_to_conv_bias = torch.zeros(out_channels)

#实例化一个”假的“3x3卷积
conv_2d_for_identity = nn.Conv2d(in_channels, out_channels, kernel_size, padding='same')
conv_2d_for_identity.weight = nn.Parameter(identity_to_conv_weight)
conv_2d_for_identity.bias = nn.Parameter(identity_to_conv_bias)
result2 = conv_2d(x) + conv_2d_for_pointwise(x) + conv_2d_for_identity(x)
print(torch.all(torch.isclose(result1,result2)))#由于是一个张量，加一个all，看每个位置是否都相等

tensor(True)


In [10]:
# 2) 融合
t3 = time.time()
conv_2d_for_fusion = nn.Conv2d(in_channels, out_channels, kernel_size, padding='same')
conv_2d_for_fusion.weight = nn.Parameter(conv_2d.weight.data + conv_2d_for_pointwise.weight.data + conv_2d_for_identity.weight.data)
conv_2d_for_fusion.bias = nn.Parameter(conv_2d.bias.data + conv_2d_for_pointwise.bias.data + conv_2d_for_identity.bias.data)
result3 = conv_2d_for_fusion(x)
t4 = time.time()
print("算子融合写法耗时：",t4-t3)
print(torch.all(torch.isclose(result2, result3)))

算子融合写法耗时： 0.0009999275207519531
tensor(True)
